In [31]:
import pandas as pd
%run heartbeat_slicer.ipynb

In [32]:
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_point_cloud

In [33]:
folder_path = "data_processed/mitdb"
record_names_path = "data_original/mitdb/RECORDS"

with open(record_names_path, 'r') as record_names_file:
    record_names = record_names_file.readlines()

record_names = [record_name[:-1] for record_name in record_names]

In [47]:
dataMitDB = []

for record_name in record_names:
    sliced = HeartbeatSlicer(folder_path + '/' + record_name + '_processed')
    dataMitDB.append(sliced.annotate_heartbeats())

In [8]:
#[patient][heartbeat_num] - [0]:heartbeat [1]:ann
dataMitDB[2][3][0].shape

(348,)

In [49]:
dataMitDB.reverse()

In [36]:
def make_point_cloud(heartbeat_):

    embedding_dimension_periodic = 3
    embedding_time_delay_periodic = 8
    embedder_periodic = SingleTakensEmbedding(
        parameters_type="fixed",
        n_jobs=2,
        time_delay=embedding_time_delay_periodic,
        dimension=embedding_dimension_periodic,
    )
    point_cloud_ = embedder_periodic.fit_transform(heartbeat)
    plot_point_cloud(point_cloud_)

    return point_cloud_


In [37]:
def make_complex_w_pers_diag(point_cloud_):
    point_cloud_ = point_cloud_[None, :, :]

    homology_dimensions = [0, 1]

    periodic_persistence = VietorisRipsPersistence(
        homology_dimensions=homology_dimensions, n_jobs=-1
    )

    diag_ = periodic_persistence.fit_transform(point_cloud_)

    return diag_


In [38]:
def make_betti_curve(diag_):
    bc = BettiCurve(n_jobs=-1)
    betti_curve_ = bc.fit_transform(diag_)
    return betti_curve_

In [89]:
# betti = make_betti_curve(dataMitDB[2][2][0])

In [90]:
# betti[0][0]

In [ ]:
df_to_be = []
patient_counter = 0
patient_number = len(dataMitDB)

for patient in dataMitDB:
    patient_counter = patient_counter+1
    print("Processing patient: ", patient_counter, "/", patient_number)
    print("---------------------------------------------")
    heartbeat_counter = 0
    heartbeat_number = len(patient)
    for heartbeat, ann in patient:
        heartbeat_counter = heartbeat_counter+1
        print("heartbeat: ", heartbeat_counter, "/", heartbeat_number)
        # Convert the segment to a point cloud
        point_cloud = make_point_cloud(heartbeat)

        # Compute the Vietoris-Rips complex and its persistence diagram
        diag = make_complex_w_pers_diag(point_cloud)

        # Compute and return Betti curve
        betti = make_betti_curve(diag)

        df_to_be.append([point_cloud, diag, betti, ann])


df = pd.DataFrame(df_to_be, columns=['points_cloud', 'diag', 'betti', 'ann'])

In [53]:
df.to_csv("data_processed/mitDB.csv")